# Regurlarization
1. Ridge (L2) - Penalty is applied on square of coefficients
2. Lasso (L1) - Penalty is applied on absolute values of coefficient

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

# Step 1 - Data Ingestion

In [3]:
import pandas as pd
df = pd.read_csv("Cars93.csv", na_values = ["","NA"], keep_default_na = False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


# Step 2 - Perform basic data quality checks

In [4]:
df.shape

(94, 28)

In [5]:
df.duplicated().sum()

np.int64(1)

In [6]:
df = df.drop_duplicates(keep="first").reset_index(drop=True)
df.shape

(93, 28)

In [7]:
m = df.isna().sum()
m[m > 0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

In [8]:
card = df.select_dtypes(include="object").nunique() / len(df)
card

Manufacturer       0.344086
Model              1.000000
Type               0.064516
AirBags            0.032258
DriveTrain         0.032258
Cylinders          0.064516
Man.trans.avail    0.021505
Origin             0.021505
Make               1.000000
dtype: float64

In [9]:
card[card >= 0.9]

Model    1.0
Make     1.0
dtype: float64

In [10]:
high_card_cols = card[card >= 0.9].index
high_card_cols

Index(['Model', 'Make'], dtype='object')

In [11]:
df = df.drop(columns = high_card_cols)
df.head()

,id,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin
0,1,Acura,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,2705,non-USA
1,2,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,3560,non-USA
2,3,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,3375,non-USA
3,4,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,3405,non-USA
4,5,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,3640,non-USA


# Step 3 - Seperate X and Y

In [12]:
X = df.drop(columns = ["id", "Weight"])
Y = df["Weight"]

In [13]:
X.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
0,Acura,Small,12.9,15.9,18.8,25,31,None,Front,4,...,Yes,13.2,5,177,102,68,37,26.5,11.0,non-USA
1,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,6,...,Yes,18.0,5,195,115,71,38,30.0,15.0,non-USA
2,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,6,...,Yes,16.9,5,180,102,67,37,28.0,14.0,non-USA
3,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,6,...,Yes,21.1,6,193,106,70,37,31.0,17.0,non-USA
4,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,4,...,Yes,21.1,4,186,109,69,39,27.0,13.0,non-USA


In [14]:
Y.head()

0    2705
1    3560
2    3375
3    3405
4    3640
Name: Weight, dtype: int64

# Step 4 - Apply train test split

In [15]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=10)

In [16]:
xtrain.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
42,Honda,Compact,13.8,17.5,21.2,24,31,Driver & Passenger,Front,4,...,Yes,17.0,4,185,107,67,41,28.0,14.0,non-USA
53,Mazda,Small,10.9,11.6,12.3,28,36,None,Front,4,...,Yes,14.5,5,172,98,66,36,26.5,13.0,non-USA
21,Chrysler,Large,29.5,29.5,29.5,20,26,NaN,Front,6,...,No,16.0,6,203,110,69,44,36.0,17.0,USA
6,Buick,Large,19.9,20.8,21.7,19,28,Driver only,Front,6,...,No,18.0,6,200,111,74,42,30.5,17.0,USA
26,Dodge,Midsize,14.8,15.6,16.4,21,27,Driver only,Front,4,...,No,16.0,6,192,105,69,42,30.5,16.0,USA


In [17]:
ytrain.head()

42    3040
53    2440
21    3570
6     3470
26    3080
Name: Weight, dtype: int64

In [18]:
xtest.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
34,Ford,Sporty,12.8,14.0,15.2,24,30,Driver only,Front,4,...,Yes,15.5,4,179,103,70,38,23.0,18.0,USA
90,Volkswagen,Sporty,22.9,23.3,23.7,18,25,None,Front,6,...,Yes,18.5,4,159,97,66,36,26.0,15.0,non-USA
3,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,6,...,Yes,21.1,6,193,106,70,37,31.0,17.0,non-USA
35,Ford,Van,14.5,19.9,25.3,15,20,Driver only,4WD,6,...,Yes,21.0,7,176,119,72,45,30.0,NaN,USA
19,Chrylser,Large,18.4,18.4,18.4,20,28,Driver & Passenger,Front,6,...,No,18.0,6,203,113,74,40,31.0,15.0,USA


In [19]:
ytest.head()

34    2710
90    2810
3     3405
35    3735
19    3515
Name: Weight, dtype: int64

In [20]:
xtrain.shape

(74, 24)

In [21]:
xtest.shape

(19, 24)

# Apply preprocessing on X

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [23]:
num_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [24]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    StandardScaler()
)

In [25]:
cat_cols = xtrain.select_dtypes(include="object").columns.tolist()
cat_cols

['Manufacturer',
 'Type',
 'AirBags',
 'DriveTrain',
 'Cylinders',
 'Man.trans.avail',
 'Origin']

In [26]:
num_cols = xtrain.select_dtypes(include="number").columns.tolist()
num_cols

['Min.Price',
 'Price',
 'Max.Price',
 'MPG.city',
 'MPG.highway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turn.circle',
 'Rear.seat.room',
 'Luggage.room']

In [27]:
pre = ColumnTransformer(
    [
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ]
).set_output(transform="pandas")

pre.fit(xtrain)

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [28]:
xtrain_pre = pre.transform(xtrain)
xtrain_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Turn.circle,num__Rear.seat.room,num__Luggage.room,cat__Manufacturer,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin
42,-0.373824,-0.208643,-0.071922,0.318308,0.363084,-0.499737,-0.070927,0.583333,0.553885,0.094849,...,0.554316,0.028926,0.031956,-0.330948,-1.373237,-1.720536,-0.180439,-0.763650,0.780189,1.114641
53,-0.699007,-0.802086,-0.851002,1.066078,1.322665,-0.871400,-0.759572,0.416667,-0.190474,-0.664766,...,-0.987766,-0.458372,-0.314235,0.266373,0.474391,1.173093,-0.180439,-0.763650,0.780189,1.114641
21,1.386650,0.998360,0.654635,-0.429463,-0.596496,0.522338,0.059357,-0.750000,-1.020721,-0.208997,...,1.479565,2.627852,1.070530,-0.928269,-0.757361,-0.273722,-0.180439,0.975122,-1.281740,-0.897150
6,0.310182,0.123283,-0.028154,-0.616405,-0.212664,0.986917,0.487434,-0.750000,-1.431073,0.398695,...,0.862732,0.841091,1.070530,-1.286661,-0.757361,-0.273722,-0.180439,0.975122,-1.281740,-0.897150
26,-0.261692,-0.399752,-0.492100,-0.242520,-0.404580,-0.220989,-0.815408,-0.750000,0.277136,-0.208997,...,0.862732,0.841091,0.724339,-0.808804,-0.141485,-0.273722,-0.180439,-0.763650,-1.281740,-0.897150


In [29]:
xtest_pre = pre.transform(xtest)
xtest_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Turn.circle,num__Rear.seat.room,num__Luggage.room,cat__Manufacturer,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin
34,-0.485956,-0.560685,-0.597145,0.318308,0.171168,-0.685568,-0.536227,0.416667,0.038559,-0.360920,...,-0.370933,-1.595402,1.416722,-0.569876,1.090267,-0.273722,-0.180439,-0.763650,0.780189,-0.897150
90,0.646578,0.374742,0.146920,-0.803348,-0.788412,0.057759,0.636330,0.916667,0.124447,0.550618,...,-0.987766,-0.620805,0.378148,1.699942,1.090267,1.173093,-0.180439,0.975122,0.780189,1.114641
3,1.532421,1.823145,1.976445,-0.616405,-0.596496,0.057759,0.524658,0.416667,0.410739,1.340618,...,-0.679350,1.003523,1.070530,-1.525589,-0.141485,-0.273722,-0.180439,0.975122,0.780189,1.114641
35,-0.295332,0.032758,0.286980,-1.364176,-1.747992,0.243590,0.022133,-0.750000,-0.457680,1.310233,...,1.787981,0.678658,0.000000,-0.569876,1.706143,-0.273722,-2.087934,0.975122,0.780189,-0.897150
19,0.141983,-0.118118,-0.317026,-0.429463,-0.212664,0.522338,0.171029,0.083333,-0.629455,0.398695,...,0.245900,1.003523,0.378148,-1.764517,-0.757361,-1.720536,-0.180439,0.975122,-1.281740,-0.897150


# Step 6 - build the base model

In [30]:
from sklearn.linear_model import LinearRegression

model1 = LinearRegression()
model1.fit(xtrain_pre, ytrain)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [31]:
model1.intercept_

np.float64(3087.567567567568)

In [32]:
model1.coef_

array([ 2.06407983e+03, -4.58780433e+03,  2.59674218e+03,  5.58440201e+01,
       -1.43419251e+02, -1.22648162e+02,  2.73991905e+02, -1.04804071e+02,
       -5.07770479e+01,  1.03696736e+02,  1.89135395e+01,  5.57991770e+01,
        2.40545292e+02, -4.43359543e+01,  2.88055727e+01, -3.01025617e+01,
        5.07497978e+01,  3.99166132e+00,  5.39157189e+01, -1.59160557e+01,
       -1.08530876e+01,  1.43701830e+01,  2.07213680e+01,  4.99623699e+01])

In [33]:
train_r2 = model1.score(xtrain_pre, ytrain)
test_r2 = model1.score(xtest_pre, ytest)
gen_err = abs(train_r2 - test_r2)
print(f"Train R2 : {train_r2:.2%}")
print(f"Test R2 : {test_r2:.2%}")
print(f"Generalization error : {gen_err:.2%}")

Train R2 : 97.25%
Test R2 : 82.27%
Generalization error : 14.98%


# Feature selection

In [34]:
from sklearn.feature_selection import SequentialFeatureSelector

lr = LinearRegression()
sel = SequentialFeatureSelector(lr, n_features_to_select="auto", direction="forward", scoring="r2")
sel.fit(xtrain_pre, ytrain)

,estimator,LinearRegression()
,n_features_to_select,'auto'
,tol,None
,direction,'forward'
,scoring,'r2'
,cv,5
,n_jobs,None
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [35]:
sel.get_feature_names_out()

array(['num__MPG.highway', 'num__Horsepower', 'num__RPM',
       'num__Rev.per.mile', 'num__Fuel.tank.capacity', 'num__Length',
       'num__Wheelbase', 'num__Luggage.room', 'cat__Manufacturer',
       'cat__Type', 'cat__Cylinders', 'cat__Origin'], dtype=object)

In [36]:
xtrain_pre_sel = sel.transform(xtrain_pre)
xtrain_pre_sel[0:5]

array([[ 0.36308441, -0.07092689,  0.58333333,  0.55388468,  0.09484922,
         0.07553707,  0.4504151 ,  0.03195613, -0.33094792, -1.3732369 ,
        -0.76364966,  1.11464086],
       [ 1.32266464, -0.75957166,  0.41666667, -0.19047443, -0.66476576,
        -0.83279622, -0.91311426, -0.3142353 ,  0.26637272,  0.47439093,
        -0.76364966,  1.11464086],
       [-0.59649582,  0.05935726, -0.75      , -1.02072113, -0.20899677,
         1.33322932,  0.90492489,  1.07053043, -0.92826856, -0.75736096,
         0.97512187, -0.89714996],
       [-0.21266373,  0.48743374, -0.75      , -1.43107295,  0.39869522,
         1.12361394,  1.05642815,  1.07053043, -1.28666095, -0.75736096,
         0.97512187, -0.89714996],
       [-0.40457977, -0.81540773, -0.75      ,  0.27713578, -0.20899677,
         0.56463961,  0.14740858,  0.724339  , -0.80880444, -0.14148501,
        -0.76364966, -0.89714996]])

In [37]:
xtest_pre_sel = sel.transform(xtest_pre)
xtest_pre_sel[0:5]

array([[ 0.17116836, -0.53622741,  0.41666667,  0.03855914, -0.36091977,
        -0.34369368, -0.15559794,  1.41672186, -0.56987618,  1.09026688,
        -0.76364966, -0.89714996],
       [-0.78841186,  0.6363299 ,  0.91666667,  0.12444673,  0.55061821,
        -1.7411295 , -1.06461752,  0.37814756,  1.69994226,  1.09026688,
         0.97512187,  1.11464086],
       [-0.59649582,  0.52465778,  0.41666667,  0.4107387 ,  1.3406178 ,
         0.6345114 ,  0.29891184,  1.07053043, -1.5255892 , -0.14148501,
         0.97512187,  1.11464086],
       [-1.74799209,  0.02213321, -0.75      , -0.45768026,  1.3102332 ,
        -0.55330905,  2.26845425,  0.        , -0.56987618,  1.70614282,
         0.97512187, -0.89714996],
       [-0.21266373,  0.17102938,  0.08333333, -0.62945544,  0.39869522,
         1.33322932,  1.35943468,  0.37814756, -1.76451746, -0.75736096,
         0.97512187, -0.89714996]])

# Model 2 after feature selection

In [38]:
model2 = LinearRegression()
model2.fit(xtrain_pre_sel, ytrain)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [39]:
train_r2 = model2.score(xtrain_pre_sel, ytrain)
test_r2 = model2.score(xtest_pre_sel, ytest)
gen_err = abs(train_r2 - test_r2)
print(f"Train R2 : {train_r2:.2%}")
print(f"Test R2 : {test_r2:.2%}")
print(f"Generalizaton Error : {gen_err:.2%}")

Train R2 : 96.60%
Test R2 : 86.81%
Generalizaton Error : 9.79%


# Ridge Regularization

In [40]:
from sklearn.linear_model import Ridge

ridge1 = Ridge(alpha=1)
ridge1.fit(xtrain_pre_sel, ytrain)

,alpha,1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [41]:
def get_gen_error(model, xtrain, ytrain, xtest, ytest):
    train_r2 = model.score(xtrain, ytrain)
    test_r2 = model.score(xtest, ytest)
    gen_err = abs(train_r2 - test_r2)
    print(f"Train R2 : {train_r2:.2%}")
    print(f"Test R2 : {test_r2:.2%}")
    print(f"Generalization Error : {gen_err:.2%}")

In [42]:
get_gen_error(ridge1, xtrain_pre_sel, ytrain, xtest_pre_sel, ytest)

Train R2 : 96.58%
Test R2 : 87.83%
Generalization Error : 8.75%


# Hyperparameter tuning on ridge model

In [43]:
params1 = {"alpha": [0.1, 0.5, 1, 10, 50, 100, 200, 500, 1000]}
print(params1)

{'alpha': [0.1, 0.5, 1, 10, 50, 100, 200, 500, 1000]}


In [44]:
from sklearn.model_selection import GridSearchCV
rr = Ridge()
gscv_ridge = GridSearchCV(rr, params1, scoring="r2", cv=5)
gscv_ridge.fit(xtrain_pre_sel, ytrain)

,estimator,Ridge()
,param_grid,"{'alpha': [0.1, 0.5, ...]}"
,scoring,'r2'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,1


In [45]:
gscv_ridge.best_params_

{'alpha': 1}

In [46]:
gscv_ridge.best_score_

np.float64(0.9487015387015474)

In [47]:
best_ridge = gscv_ridge.best_estimator_
best_ridge

,alpha,1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [48]:
get_gen_error(best_ridge, xtrain_pre_sel, ytrain, xtest_pre_sel, ytest)

Train R2 : 96.58%
Test R2 : 87.83%
Generalization Error : 8.75%


# Add polynomial features
col^2, col^3, ...

In [49]:
from sklearn.preprocessing import PolynomialFeatures

ridge_poly = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    Ridge()
)

params2 = {"ridge__alpha": [0.1, 0.5, 1, 10, 50, 100, 200, 500, 1000]}

gscv_ridge_poly = GridSearchCV(ridge_poly, params2, scoring="r2", cv=5)
gscv_ridge_poly.fit(xtrain_pre_sel, ytrain)

,estimator,"Pipeline(step...e', Ridge())])"
,param_grid,"{'ridge__alpha': [0.1, 0.5, ...]}"
,scoring,'r2'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,2


In [50]:
gscv_ridge_poly.best_params_

{'ridge__alpha': 10}

In [51]:
gscv_ridge_poly.best_score_

np.float64(0.9360431987472773)

In [52]:
best_ridge_poly = gscv_ridge_poly.best_estimator_
best_ridge_poly

,steps,"[('polynomialfeatures', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,degree,2
,interaction_only,False
,include_bias,False
,order,'C'
,alpha,10
,fit_intercept,True
,copy_X,True


In [53]:
get_gen_error(best_ridge_poly, xtrain_pre_sel, ytrain, xtest_pre_sel, ytest)

Train R2 : 98.78%
Test R2 : 85.19%
Generalization Error : 13.58%


# Lasso

In [54]:
from sklearn.linear_model import Lasso

ls = Lasso()
params3 = {"alpha": [0.1, 0.5, 1, 10, 50, 100, 200, 500, 1000]}

gscv_lasso = GridSearchCV(ls, params3, scoring="r2", cv=5)
gscv_lasso.fit(xtrain_pre_sel, ytrain)

,estimator,Lasso()
,param_grid,"{'alpha': [0.1, 0.5, ...]}"
,scoring,'r2'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,1


In [55]:
gscv_lasso.best_params_

{'alpha': 1}

In [56]:
gscv_lasso.best_score_

np.float64(0.9486287596563983)

In [57]:
best_lasso = gscv_lasso.best_estimator_
best_lasso

,alpha,1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [58]:
get_gen_error(best_lasso, xtrain_pre_sel, ytrain, xtest_pre_sel, ytest)

Train R2 : 96.59%
Test R2 : 87.90%
Generalization Error : 8.69%


# Apply Lasso with polynomial features

In [59]:
lasso_poly = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    Lasso()
)

params3 = {"lasso__alpha": [0.1, 0.5, 1, 10, 50, 100, 200, 500, 1000]}

gscv_poly_lasso = GridSearchCV(lasso_poly, params3, scoring="r2", cv=5)
gscv_poly_lasso.fit(xtrain_pre_sel, ytrain)

,estimator,"Pipeline(step...o', Lasso())])"
,param_grid,"{'lasso__alpha': [0.1, 0.5, ...]}"
,scoring,'r2'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,2


In [60]:
gscv_poly_lasso.best_params_

{'lasso__alpha': 10}

In [61]:
gscv_poly_lasso.best_score_

np.float64(0.9340508678778852)

In [62]:
best_poly_lasso = gscv_poly_lasso.best_estimator_
best_poly_lasso

,steps,"[('polynomialfeatures', ...), ('lasso', ...)]"
,transform_input,None
,memory,None
,verbose,False
,degree,2
,interaction_only,False
,include_bias,False
,order,'C'
,alpha,10
,fit_intercept,True
,precompute,False


In [63]:
get_gen_error(best_poly_lasso, xtrain_pre_sel, ytrain, xtest_pre_sel, ytest)

Train R2 : 97.66%
Test R2 : 80.25%
Generalization Error : 17.41%


# From above it is clear that the Lasso model has good test r2 and low generalization error

In [64]:
best_lasso

,alpha,1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [65]:
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

def evaluate_model(model, x, y):
    ypred = model.predict(x)
    rmse = root_mean_squared_error(y, ypred)
    mae = mean_absolute_error(y, ypred)
    mape = mean_absolute_percentage_error(y, ypred)
    r2 = r2_score(y, ypred)
    print(f"RMSE : {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"MAPE : {mape:.2%}")
    print(f"R2 : {r2:.2%}")

In [66]:
print("Train Results :")
evaluate_model(best_lasso, xtrain_pre_sel, ytrain)

Train Results :
RMSE : 112.67
MAE : 88.49
MAPE : 2.94%
R2 : 96.59%


In [67]:
print("Test Results :")
evaluate_model(best_lasso, xtest_pre_sel, ytest)

Test Results :
RMSE : 167.70
MAE : 131.43
MAPE : 4.44%
R2 : 87.90%


# Out of sample prediction

In [68]:
xnew = pd.read_csv("sample.csv", na_values=["", "NA"], keep_default_na=False)
xnew

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,15.0,6,190,106,65,37,31.0,17.0,non-USA,Audi 100
1,Pontiac,Sunbird,Compact,9.4,11.1,12.8,23,31,None,Front,...,15.2,5,181,101,66,39,25.0,13.0,USA,Pontiac Sunbird
2,Chevrolet,Lumina,Midsize,13.4,15.9,18.4,21,29,None,Front,...,16.5,6,198,108,71,40,28.5,16.0,USA,Chevrolet Lumina
3,Mazda,RX-7,Sporty,32.5,32.5,32.5,17,25,Driver only,Rear,...,20.0,2,169,96,69,37,NaN,NaN,non-USA,Mazda RX-7
4,Volkswagen,Fox,Small,8.7,9.1,9.5,25,33,None,Front,...,12.4,4,163,93,63,34,26.0,10.0,non-USA,Volkswagen Fox


In [69]:
xnew_pre = pre.transform(xnew)
xnew_pre

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Turn.circle,num__Rear.seat.room,num__Luggage.room,cat__Manufacturer,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin
0,1.532421,1.823145,1.976445,-0.616405,-0.596496,0.057759,0.524658,0.416667,0.410739,-0.512843,...,-0.679350,1.003523,1.070530,-1.525589,-0.141485,-0.273722,-0.180439,0.975122,0.780189,1.114641
1,-0.867205,-0.852378,-0.807234,0.131365,0.363084,-0.685568,-0.629288,-0.083333,0.658858,-0.452074,...,-0.062517,-0.945671,-0.314235,0.983157,-1.373237,1.173093,-0.180439,-0.763650,0.780189,-0.897150
2,-0.418677,-0.369577,-0.317026,-0.242520,-0.020748,-0.499737,-0.629288,-0.083333,0.525255,-0.057074,...,0.245900,0.191359,0.724339,-1.047733,-0.141485,1.173093,-0.180439,-0.763650,-1.281740,-0.897150
3,1.723046,1.300111,0.917246,-0.990291,-0.788412,-1.335979,2.069456,2.083333,0.009930,1.006387,...,-0.679350,0.000000,0.000000,0.266373,1.090267,-0.273722,1.727057,-2.502421,0.780189,1.114641
4,-0.945698,-1.053545,-1.096106,0.505250,0.746917,-0.871400,-1.169036,0.416667,0.439368,-1.302842,...,-1.604599,-0.620805,-1.352810,1.699942,0.474391,1.173093,-0.180439,-0.763650,0.780189,1.114641


In [70]:
xnew_pre_sel = sel.transform(xnew_pre)
xnew_pre_sel

array([[-0.59649582,  0.52465778,  0.41666667,  0.4107387 , -0.51284277,
         0.42489603,  0.29891184,  1.07053043, -1.5255892 , -0.14148501,
         0.97512187,  1.11464086],
       [ 0.36308441, -0.62928752, -0.08333333,  0.6588584 , -0.45207357,
        -0.20395009, -0.45860447, -0.3142353 ,  0.98315749, -1.3732369 ,
        -0.76364966, -0.89714996],
       [-0.02074768, -0.62928752, -0.08333333,  0.52525548, -0.05707377,
         0.98387036,  0.60191837,  0.724339  , -1.04773269, -0.14148501,
        -0.76364966, -0.89714996],
       [-0.78841186,  2.06945551,  2.08333333,  0.00992995,  1.00638721,
        -1.04241159, -1.21612078,  0.        ,  0.26637272,  1.09026688,
        -2.50242119,  1.11464086],
       [ 0.7469165 , -1.16903612,  0.41666667,  0.43936789, -1.30284235,
        -1.46164234, -1.67063057, -1.3528096 ,  1.69994226,  0.47439093,
        -0.76364966,  1.11464086]])

In [71]:
preds = best_lasso.predict(xnew_pre_sel)
preds

array([3303.50667802, 2667.27979354, 3107.16174246, 3312.25585789,
       2269.80066693])

In [72]:
xnew["Weight_pred"] = preds.round(2)
xnew

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make,Weight_pred
0,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,6,190,106,65,37,31.0,17.0,non-USA,Audi 100,3303.51
1,Pontiac,Sunbird,Compact,9.4,11.1,12.8,23,31,None,Front,...,5,181,101,66,39,25.0,13.0,USA,Pontiac Sunbird,2667.28
2,Chevrolet,Lumina,Midsize,13.4,15.9,18.4,21,29,None,Front,...,6,198,108,71,40,28.5,16.0,USA,Chevrolet Lumina,3107.16
3,Mazda,RX-7,Sporty,32.5,32.5,32.5,17,25,Driver only,Rear,...,2,169,96,69,37,NaN,NaN,non-USA,Mazda RX-7,3312.26
4,Volkswagen,Fox,Small,8.7,9.1,9.5,25,33,None,Front,...,4,163,93,63,34,26.0,10.0,non-USA,Volkswagen Fox,2269.80


In [73]:
xnew.to_csv("results.csv", index=False)

# Save pre, sel, model 

In [74]:
pre

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [75]:
sel

,estimator,LinearRegression()
,n_features_to_select,'auto'
,tol,None
,direction,'forward'
,scoring,'r2'
,cv,5
,n_jobs,None
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [76]:
best_lasso

,alpha,1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [77]:
import joblib
joblib.dump(pre, "pre.joblib")

['pre.joblib']

In [78]:
joblib.dump(sel, "sel.joblib")

['sel.joblib']

In [79]:
joblib.dump(best_lasso, "lasso.joblib")

['lasso.joblib']

# Load model objects

In [80]:
p = joblib.load("pre.joblib")
p

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [81]:
s = joblib.load("sel.joblib")
s

,estimator,LinearRegression()
,n_features_to_select,'auto'
,tol,None
,direction,'forward'
,scoring,'r2'
,cv,5
,n_jobs,None
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [82]:
m = joblib.load("lasso.joblib")
m

,alpha,1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'
